## Fandom Radius National Graph

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px


In [2]:
# Load all team radii
radii_df = pd.read_csv("/Users/elisabethkollrack/Thesis/EK-Thesis/Fandom Radii/fandom_radius_all_teams.csv")

In [3]:
# Geodesic circle function
def geodesic_circle(lat, lon, radius_km, n_points=360):
    earth_radius_km = 6371.0
    angles = np.linspace(0, 2*np.pi, n_points)

    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)

    circle_lats = np.arcsin(
        np.sin(lat_rad) * np.cos(radius_km / earth_radius_km) +
        np.cos(lat_rad) * np.sin(radius_km / earth_radius_km) * np.cos(angles)
    )

    circle_lons = lon_rad + np.arctan2(
        np.sin(angles) * np.sin(radius_km / earth_radius_km) * np.cos(lat_rad),
        np.cos(radius_km / earth_radius_km) - np.sin(lat_rad) * np.sin(circle_lats)
    )

    return np.degrees(circle_lats), np.degrees(circle_lons)

In [4]:
overall_df = radii_df[radii_df["season"] == "overall"].copy()

fig = px.scatter_geo(
    overall_df,
    lat="lat",
    lon="lon",
    hover_name="city",
    scope="usa",
    title="NFL Fandom Radii Across the United States (2011–2014 Overall)"
)

for _, row in overall_df.iterrows():
    lats, lons = geodesic_circle(row["lat"], row["lon"], row["radius_km"])

    trace = px.line_geo(lat=lats, lon=lons).data[0]

    trace.update(
        hovertemplate=(
            f"<b>{row['team_name']}</b><br>"
            f"Fandom radius ≈ {int(row['radius_km'])} km"
            "<extra></extra>"
        ),
        name=row["team_name"],
        showlegend=False,
        opacity=0.4
    )

    fig.add_trace(trace)

    fig.update_geos(
    scope="usa",
    showland=True,
    landcolor="lightgray",
    showocean=True,
    oceancolor="azure",
    showlakes=True,
    lakecolor="azure"
)

fig.update_layout(title_x=0.5)
fig.show()



In [5]:
fig_hist = px.bar(
    overall_df.sort_values("radius_km", ascending=True),
    x="radius_km",
    y="team_name",
    orientation="h",
    title="NFL Fandom Radii by Team (2011–2014)",
    labels={"R_km": "Fandom Radius (km)", "team": "NFL Team"}
)

fig_hist.update_layout(
    title_x=0.5,
    margin=dict(l=220, t=80),
    font=dict(size=10)
)

fig_hist.update_yaxes(
    tickfont=dict(size=9),
    type="category",
    tickmode="linear"
)

fig_hist.show()
